# 图像特征练习
我们已经看到，通过在输入图像的像素上训练线性分类器，我们可以在图像分类任务上实现合理的表现。在本练习中，我们将展示可以通过不在原始像素上训练线性分类器，而是在根据原始像素计算的特征上训练来提高分类性能。
你为这个练习所做的所有工作都将在这个文件上完成。

In [ ]:
import random
import numpy as np
from dlcv.data_utils import load_CIFAR10
import matplotlib.pyplot as plt


%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Load data
与之前的练习类似，我们将从磁盘加载CIFAR-10数据。

In [ ]:
from dlcv.features import color_histogram_hsv, hog_feature

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000):
    # Load the raw CIFAR-10 data
    cifar10_dir = 'dlcv/datasets/cifar-10-batches-py'

    # Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
    try:
       del X_train, y_train
       del X_test, y_test
       print('Clear previously loaded data.')
    except:
       pass

    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # Subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    
    return X_train, y_train, X_val, y_val, X_test, y_test

X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()

## 提取特征
对于每个图像，我们将使用HSV颜色空间中的色调通道计算定向梯度直方图（HOG）以及颜色直方图。我们通过连接HOG和颜色直方图的特征向量来形成每个图像的最终特征向量。

粗略地说，HOG应该在忽略颜色信息的情况下捕捉图像的纹理，而颜色直方图表示输入图像的颜色，而忽略纹理。因此，我们预计两者一起使用应该比单独使用任何一个效果更好。为了自己的利益，验证这个假设是一件好事。

“hog_feature”和“color_histogram_hsv”函数都对单个图像进行操作，并返回该图像的特征向量。extract_features函数获取一组图像和一组特征函数，并评估每个图像上的每个特征函数，将结果存储在矩阵中，其中每列是单个图像的所有特征向量的级联。

In [ ]:
from dlcv.features import *

num_color_bins = 10 # Number of bins in the color histogram
feature_fns = [hog_feature, lambda img: color_histogram_hsv(img, nbin=num_color_bins)]
X_train_feats = extract_features(X_train, feature_fns, verbose=True)
X_val_feats = extract_features(X_val, feature_fns)
X_test_feats = extract_features(X_test, feature_fns)

# 预处理：减去平均特征
mean_feat = np.mean(X_train_feats, axis=0, keepdims=True)
X_train_feats -= mean_feat
X_val_feats -= mean_feat
X_test_feats -= mean_feat

# 预处理：除以标准差。这样可以确保每个特征具有大致相同的比例。
std_feat = np.std(X_train_feats, axis=0, keepdims=True)
X_train_feats /= std_feat
X_val_feats /= std_feat
X_test_feats /= std_feat

# 预处理：添加偏差标注
X_train_feats = np.hstack([X_train_feats, np.ones((X_train_feats.shape[0], 1))])
X_val_feats = np.hstack([X_val_feats, np.ones((X_val_feats.shape[0], 1))])
X_test_feats = np.hstack([X_test_feats, np.ones((X_test_feats.shape[0], 1))])

## 基于特征训练SVM
使用任务早期开发的多类SVM代码，在上面提取的特征之上训练SVM；这应该比直接在原始像素之上训练SVM获得更好的结果。

In [ ]:
# 使用验证集调整学习率和正则化强度

from dlcv.classifiers.linear_classifier import LinearSVM

learning_rates = np.arange(3e-8,5e-7,2e-8)
regularization_strengths = np.arange(1e4,2e5,2e4)

results = {}
best_val = -1
best_svm = None

################################################################################
# TODO:
# 使用验证集设置学习率和正则化强度
# 这应该与您对SVM所做的验证相同；将经过最佳训练的分类器保存在best_svm中。
# 您可能还想在颜色直方图中使用不同数量的数值条。
# 如果你小心的话，你应该能够在验证集上获得接近0.44的准确度。
################################################################################
# *****     代码的开头（不要删除/修改此行）        *****

pass

# *****     代码的结尾（不要删除/修改此行）        *****

# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved: %f' % best_val)

In [ ]:
# 在测试集上评估你训练过的SVM：你应该能够得到至少0.40
y_test_pred = best_svm.predict(X_test_feats)
test_accuracy = np.mean(y_test == y_test_pred)
print(test_accuracy)

In [ ]:
# 获得关于算法如何工作的直觉的一个重要方法是可视化它所犯的错误。
# 在这个可视化中，我们展示了当前系统错误分类的图像示例。
# 第一列显示了我们的系统标记为“平面”的图像，但其真正的标签不是“平面”。

examples_per_class = 8
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for cls, cls_name in enumerate(classes):
    idxs = np.where((y_test != cls) & (y_test_pred == cls))[0]
    idxs = np.random.choice(idxs, examples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt.subplot(examples_per_class, len(classes), i * len(classes) + cls + 1)
        plt.imshow(X_test[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls_name)
plt.show()

### Inline question 1:
描述您看到的错误分类结果。它们有意义吗？


$\color{blue}{\textit Your Answer:}$




## 图像特征的神经网络
在本任务的早期，我们看到在原始像素上训练两层神经网络比在原始像素中训练线性分类器获得了更好的分类性能。在这次实验中，我们看到图像特征上的线性分类器优于原始像素上的线性分类。
为了保证完整性，我们还应该尝试在图像特征上训练神经网络。这种方法应该优于以前的所有方法：你应该能够很容易地在测试集上实现超过55%的分类准确率；我们的最佳模型实现了大约60%的分类准确率。

In [ ]:
# 预处理：删除偏差尺寸
# Make sure to run this cell only ONCE
print(X_train_feats.shape)
X_train_feats = X_train_feats[:, :-1]
X_val_feats = X_val_feats[:, :-1]
X_test_feats = X_test_feats[:, :-1]

print(X_train_feats.shape)

In [ ]:
!pip install --upgrade pip
!pip install future

In [ ]:
from dlcv.classifiers.fc_net import TwoLayerNet
from dlcv.solver import Solver

input_dim = X_train_feats.shape[1]
hidden_dim = 500
num_classes = 10

data = {
    'X_train': X_train_feats,
    'y_train': y_train,
    'X_val': X_val_feats,
    'y_val': y_val,
    'X_test': X_test_feats,
    'y_test': y_test,
}
# net=TwoLayerNet(input_dim,hidden_dim,reg=0.001)
# print(net.params['b1'][:5])
# solver=Solver(net,data,update_rule='adam')
# solver.train()
learning_rate=np.arange(8e-5,9e-5,2e-6)
num_epochs=15
regularization_strengths = np.arange(1e-8,2e-7,3e-8)
best_net = None
best_acc=-1

################################################################################
# TODO: 在图像特征上训练两层神经网络。您可能需要像前几节中那样交叉验证各种参数。将您的最佳模型存储在best_net变量中。
################################################################################
# *****     代码的开头（不要删除/修改此行）        *****

pass

# *****     代码的结尾（不要删除/修改此行）        *****


In [ ]:
# 在测试集上运行你最好的神经网络分类器。你应该能够获得超过55%的准确率。

y_test_pred = np.argmax(best_net.loss(data['X_test']), axis=1)
test_acc = (y_test_pred == data['y_test']).mean()
print(test_acc)